In [ ]:
#gerekli kütüphaneleri indiriyoruz
%pip install pandas scikit-learn
%pip install matplotlib

In [ ]:
#test ve train setlerimizi import ediyoruz
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.read_csv("evler-train.csv")
test = pd.read_csv("evler-test.csv")

Verilerin yapısı "evNo,fiyat,brutM2,netM2,oda,salon,yas,kat,katMaks,dogalGaz,banyo,amerikan,balkon,asansor,otopark,esya,siteMi,aidat,guneyMi" şeklinde.

In [ ]:
#4 farklı regresyon modeli deniyoruz
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [ ]:
#lineer regresyon modeli
lr_model = LinearRegression()
lr_model.fit(train.drop(columns=["fiyat", "evNo"]), train["fiyat"])
lr_predictions = lr_model.predict(test.drop(columns=["fiyat", "evNo"]))

In [ ]:
#DecisionTreeRegressor modeli
dt_model = DecisionTreeRegressor()
dt_model.fit(train.drop(columns=["fiyat", "evNo"]), train["fiyat"])
dt_predictions = dt_model.predict(test.drop(columns=["fiyat", "evNo"]))

In [ ]:
#RandomForestRegressor modeli
rf_model = RandomForestRegressor()
rf_model.fit(train.drop(columns=["fiyat", "evNo"]), train["fiyat"])
rf_predictions = rf_model.predict(test.drop(columns=["fiyat", "evNo"]))

In [ ]:
#SVR modeli
svr_model = SVR()
svr_model.fit(train.drop(columns=["fiyat", "evNo"]), train["fiyat"])
svr_predictions = svr_model.predict(test.drop(columns=["fiyat", "evNo"]))

In [ ]:
#4 farklı metrik ile modellerimizi değerlendiriyoruz
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, median_absolute_error
def evaluate_model(true_values, predictions):
    mae = mean_absolute_error(true_values, predictions)
    mse = mean_squared_error(true_values, predictions)
    r2 = r2_score(true_values, predictions)
    medae = median_absolute_error(true_values, predictions)
    return {"MAE": mae, "MSE": mse, "R2": r2, "MedAE": medae}
lr_results = evaluate_model(test["fiyat"], lr_predictions)
print("Linear Regression Results:", lr_results)
dt_results = evaluate_model(test["fiyat"], dt_predictions)
print("Decision Tree Results:", dt_results)
rf_results = evaluate_model(test["fiyat"], rf_predictions)
print("Random Forest Results:", rf_results)
svr_results = evaluate_model(test["fiyat"], svr_predictions)
print("SVR Results:", svr_results)

In [ ]:
#sonuçları görselleştiriyoruz
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(test["fiyat"].values, label="Gerçek Fiyatlar", color="blue")
plt.plot(lr_predictions, label="LR Tahminleri", color="red")
plt.plot(dt_predictions, label="DT Tahminleri", color="green")
plt.plot(rf_predictions, label="RF Tahminleri", color="orange")
plt.plot(svr_predictions, label="SVR Tahminleri", color="purple")
plt.title("Gerçek ve Tahmin Edilen Fiyatlar")
plt.xlabel("Örnekler")
plt.ylabel("Fiyat")
plt.legend()
plt.show()

In [ ]:
#sonuçları karşılaştırarak en iyi modeli seçiyoruz
results = {
    "Linear Regression": lr_results,
    "Decision Tree": dt_results,
    "Random Forest": rf_results,
    "SVR": svr_results
}
best_model = min(results, key=lambda x: results[x]["MAE"])
print("En İyi Model:", best_model, "with MAE:", results[best_model]["MAE"])